In [201]:
#from __future__ import unicode_literals, print_function, division
import numpy as np
import sys
import math
import csv
import sys
sys.maxsize

9223372036854775807

In [202]:
activate_codes_num = -1
next_k_step = 1
training_chunk = 0
test_chunk = 1
#input_size = 100
topk = 10

#Top-n frequent (TopFreq): It uses the most frequent s items
#that appear in all the baskets of the training data as the
#predicted next baskets for all persons.

In [203]:
from sklearn.neighbors import NearestNeighbors

In [204]:
def add_history(data_history,training_key_set,output_size):
    sum_history = {}
    for key in training_key_set:
        sum_vector = np.zeros(output_size)
        count = 0
        for lst in data_history[key]:
            vec = np.zeros(output_size)
            for ele in lst:
                vec[ele] = 1
            if vec[-2] == 1 or vec[-1] == 1:
                continue
            sum_vector += vec
            count += 1
        sum_vector = sum_vector / count
        sum_history[key] = sum_vector
    return sum_history

In [205]:
def temporal_decay_add_history(data_set, key_set, output_size,within_decay_rate):
    sum_history = {}
    for key in key_set:
        vec_list = data_set[key]
        num_vec = len(vec_list) - 2
        his_list = np.zeros(output_size)
        for idx in range(1,num_vec+1):
            his_vec = np.zeros(output_size)
            decayed_val = np.power(within_decay_rate,num_vec-idx)
            for ele in vec_list[idx]:
                his_vec[ele] = decayed_val
            his_list += his_vec
        sum_history[key] = his_list/num_vec
        # sum_history[key] = np.multiply(his_list / num_vec, IDF)

    return sum_history

In [206]:
def KNN(query_set, target_set, k):
    history_mat = []
    for key in target_set.keys():
        history_mat.append(target_set[key])
    test_mat = []
    for key in query_set.keys():
        test_mat.append(query_set[key])
    # print('Finding k nearest neighbors...')
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(history_mat)
    distances, indices = nbrs.kneighbors(test_mat)
    # print('Finish KNN search.' )
    return indices,distances

In [207]:
def weighted_aggragate_outputs(data_chunk,training_key_set,index,distance,output_size):
    output_vectors = []
    key_set = training_key_set
    for index_list_id in range(len(index)):
        outputs = []
        for vec_idx in range(1,next_k_step+1):

            target_vec_list = []
            weight_list = []
            for id in range(len(index[index_list_id])):
                dis = distance[index_list_id][id]
                if dis == 0:
                    weight_list.append(0)
                else:
                    weight_list.append(1 / dis)
            new_weight = softmax(weight_list)
            for i in range(len(new_weight)):
                if new_weight[i] == 0:
                    new_weight[i] = 1
            vec = np.zeros(output_size)
            for id in range(len(index[index_list_id])):
                idx = index[index_list_id][id]
                target_list = data_chunk[test_chunk][key_set[idx]][vec_idx]
                for ele in target_list:
                    vec[ele] += new_weight[id]
            outputs.append(vec)
        output_vectors.append(outputs)
    return output_vectors

In [208]:
def KNN_history_record1(sum_history, output_size,k):
    history_mat = []
    for key in sum_history.keys():
        history_mat.append(sum_history[key])

    print('Finding k nearest neighbors...')
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(history_mat)
    distances, indices = nbrs.kneighbors(history_mat)
    KNN_history = {}
    key_set = list(sum_history)
    for id in range(len(key_set)):
#    for idx_list in indices:
        idx_list = indices[id]
        NN_history = np.zeros(output_size)
        for idx in idx_list:
            NN_history += sum_history[key_set[idx]]
        NN_history = NN_history / k
        KNN_history[key_set[id]] = NN_history
        return KNN_history

In [209]:
def KNN_history_record2(query_set, sum_history, output_size,k):
    history_mat = []
    for key in sum_history.keys():
        history_mat.append(sum_history[key])
    test_mat = []
    for key in query_set.keys():
        test_mat.append(query_set[key])
    print('Finding k nearest neighbors...')
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(history_mat)
    distances, indices = nbrs.kneighbors(test_mat)
    KNN_history = {}
    key_set = list(query_set)
    training_key_set = list(sum_history)
    for id in range(len(key_set)):
#    for idx_list in indices:
        idx_list = indices[id]
        NN_history = np.zeros(output_size)
        for idx in idx_list:
            NN_history += sum_history[training_key_set[idx]]
        NN_history = NN_history / k
        KNN_history[key_set[id]] = NN_history
    return KNN_history,indices

In [210]:
def group_history_list(his_list,group_size):
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        return grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:

            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        return grouped_vec_list, group_size

In [211]:
def temporal_decay_sum_history(data_set, key_set, output_size,group_size,within_decay_rate,group_decay_rate):
    sum_history = {}
    for key in key_set:
        vec_list = data_set[key]
        num_vec = len(vec_list) - 2
        his_list = []
        for idx in range(1,num_vec+1):
            his_vec = np.zeros(output_size)
            decayed_val = np.power(within_decay_rate,num_vec-idx)
            for ele in vec_list[idx]:
                his_vec[ele] = decayed_val
            his_list.append(his_vec)

        grouped_list,real_group_size = group_history_list(his_list,group_size)
        his_vec = np.zeros(output_size)
        for idx in range(real_group_size):
            decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
            if idx>=len(grouped_list):
                print( 'idx: '+ str(idx))
                print('len(grouped_list): ' + str(len(grouped_list)))
            his_vec += grouped_list[idx]*decayed_val
        sum_history[key] = his_vec/real_group_size
        # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)
    return sum_history

In [212]:
def partition_the_data(data_chunk,key_set):
    filtered_key_set = []
    for key in key_set:
        if len(data_chunk[training_chunk][key])<=3:
            continue
        if len(data_chunk[test_chunk][key])<2+next_k_step:
            continue
        filtered_key_set.append(key)

    training_key_set = filtered_key_set[0:int(4 / 5 * len(filtered_key_set))]
    print(len(training_key_set))
    test_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)):]
    return training_key_set,test_key_set

In [213]:
def partition_the_data_validate(data_chunk, key_set, next_k_step):
    filtered_key_set = []
    past_chunk = 0
    future_chunk = 1
    for key in key_set:
        if len(data_chunk[past_chunk][key]) <= 3:
            continue
        if len(data_chunk[future_chunk][key]) < 2 + next_k_step:
            continue
        filtered_key_set.append(key)

    training_key_set = filtered_key_set[0:int(4 / 5 * len(filtered_key_set)*0.9)]
    validation_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)*0.9):int(4 / 5 * len(filtered_key_set))]
    print('Number of training instances: ' + str(len(training_key_set)))
    test_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)):]
    return training_key_set, validation_key_set, test_key_set

In [214]:
def most_frequent_elements(data_chunk,index,training_key_set,output_size):
    output_vectors = []

    for vec_idx in range(1,next_k_step+1):
        vec = np.zeros(output_size)
        for idx in index:
            target_vec = data_chunk[test_chunk][training_key_set[idx]][vec_idx]
            for ele in target_vec:
                vec[ele] += 1

        output_vectors.append(vec)
    return output_vectors

In [215]:
def predict_with_elements_in_input(sum_history,key):
    output_vectors = []

    for idx in range(next_k_step):
        vec = sum_history[key]
        output_vectors.append(vec)
    return output_vectors

In [216]:
def generate_dictionary_BA(files, attributes_list):
    path = ''
    #files = ['Coborn_history_order.csv','Coborn_future_order.csv']
    #files = ['BA_history_order.csv', 'BA_future_order.csv']
    #attributes_list = ['MATERIAL_NUMBER']
    dictionary_table = {}
    counter_table = {}
    for attr in attributes_list:
        dictionary = {}
        dictionary_table[attr] = dictionary
        counter_table[attr] = 0

    #csv.field_size_limit(sys.maxsize)
    for filename in files:
        count = 0
        with open(path + filename, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in reader:
                if count == 0:
                    count += 1
                    continue
                key = attributes_list[0]
                if row[2] not in dictionary_table[key]:
                    dictionary_table[key][row[2]] = counter_table[key]
                    counter_table[key] = counter_table[key] + 1
                    count += 1
    print(counter_table)

    total = 0
    for key in counter_table.keys():
        total = total + counter_table[key]

    print('# dimensions of final vector: ' + str(total) + ' | '+str(count-1))

    return dictionary_table, total, counter_table

In [217]:
def read_claim2vector_embedding_file_no_vector(files):
    #attributes_list = ['DRG', 'PROVCAT ', 'RVNU_CD', 'DIAG', 'PROC']
    attributes_list = ['MATERIAL_NUMBER']
    path = ''
    print('start dictionary generation...')
    dictionary_table, num_dim, counter_table = generate_dictionary_BA(files, attributes_list)
    print('finish dictionary generation*****')
    usr_attr = 'CUSTOMER_ID'
    ord_attr = 'ORDER_NUMBER'

    #dictionary_table, num_dim, counter_table = GDF.generate_dictionary(attributes_list)

    freq_max = 200
    ## all the follow three ways array. First index is patient, second index is the time step, third is the feature vector
    data_chunk = []
    day_gap_counter = []
    claims_counter = 0
    num_claim = 0
    code_freq_at_first_claim = np.zeros(num_dim+2)


    for file_id in range(len(files)):

        count = 0
        data_chunk.append({})
        filename = files[file_id]
        with open(path + filename, 'r') as csvfile:
            #gap_within_one_year = np.zeros(365)
            reader = csv.DictReader(csvfile)
            last_pid_date = '*'
            last_pid = '-1'
            last_days = -1
            # 2 more elements in the end for start and end states
            feature_vector = []
            for row in reader:
                cur_pid_date = row[usr_attr] + '_' + row[ord_attr]
                cur_pid = row[usr_attr]
                #cur_days = int(row[ord_attr])

                if cur_pid != last_pid:
                    # start state
                    tmp = [-1]
                    data_chunk[file_id][cur_pid] = []
                    data_chunk[file_id][cur_pid].append(tmp)
                    num_claim = 0

                if cur_pid_date not in last_pid_date:
                    if last_pid_date not in '*' and last_pid not in '-1':
                        sorted_feature_vector = np.sort(feature_vector)
                        data_chunk[file_id][last_pid].append(sorted_feature_vector)
                        if len(sorted_feature_vector) > 0:
                            count = count + 1
                        #data_chunk[file_id][last_pid].append(feature_vector)
                    feature_vector = []

                    claims_counter = 0
                if cur_pid != last_pid:
                    # end state
                    if last_pid not in '-1':

                        tmp = [-1]
                        data_chunk[file_id][last_pid].append(tmp)

                key = attributes_list[0]

                within_idx = dictionary_table[key][row[key]]
                previous_idx = 0

                for j in range(attributes_list.index(key)):
                    previous_idx = previous_idx + counter_table[attributes_list[j]]
                idx = within_idx + previous_idx

                # set corresponding dimention to 1
                if idx not in feature_vector:
                    feature_vector.append(idx)

                last_pid_date = cur_pid_date
                last_pid = cur_pid
                #last_days = cur_days
                if file_id == 1:
                    claims_counter = claims_counter + 1


            if last_pid_date not in '*' and last_pid not in '-1':
                data_chunk[file_id][last_pid].append(np.sort(feature_vector))

    return data_chunk, num_dim + 2, code_freq_at_first_claim

In [218]:
def get_precision_recall_Fscore(groundtruth,pred):
    a = groundtruth
    b = pred
    correct = 0
    truth = 0
    positive = 0

    for idx in range(len(a)):
        if a[idx] == 1:
            truth += 1
            if b[idx] == 1:
                correct += 1
        if b[idx] == 1:
            positive += 1

    flag = 0
    if 0 == positive:
        precision = 0
        flag = 1
        #print('postivie is 0')
    else:
        precision = correct/positive
    if 0 == truth:
        recall = 0
        flag = 1
        #print('recall is 0')
    else:
        recall = correct/truth

    if flag == 0 and precision + recall > 0:
        F = 2*precision*recall/(precision+recall)
    else:
        F = 0
    return precision, recall, F, correct

In [219]:
def get_F_score(prediction, test_Y):
    jaccard_similarity = []
    prec = []
    rec = []

    count = 0
    for idx in range(len(test_Y)):
        pred = prediction[idx]
        T = 0
        P = 0
        correct = 0
        for id in range(len(pred)):
            if test_Y[idx][id] == 1:
                T = T + 1
                if pred[id] == 1:
                    correct = correct + 1
            if pred[id] == 1:
                P = P + 1

        if P == 0 or T == 0:
            continue
        precision = correct / P
        recall = correct / T
        prec.append(precision)
        rec.append(recall)
        if correct == 0:
            jaccard_similarity.append(0)
        else:
            jaccard_similarity.append(2 * precision * recall / (precision + recall))
        count = count + 1

    print(
        'average precision: ' + str(np.mean(prec)))
    print('average recall : ' + str(
        np.mean(rec)))
    print('average F score: ' + str(
        np.mean(jaccard_similarity)))

In [220]:
def get_DCG(groundtruth, pred_rank_list,k):
    count = 0
    dcg = 0
    for pred in pred_rank_list:
        if count >= k:
            break
        if groundtruth[pred] == 1:
            dcg += (1)/math.log2(count+1+1)
        count += 1

    return dcg

In [221]:
def get_NDCG1(groundtruth, pred_rank_list,k):
    count = 0
    dcg = 0
    for pred in pred_rank_list:
        if count >= k:
            break
        if groundtruth[pred] == 1:
            dcg += (1)/math.log2(count+1+1)
        count += 1
    idcg = 0
    num_real_item = np.sum(groundtruth)
    num_item = int(num_real_item)
    for i in range(num_item):
        idcg += (1) / math.log2(i + 1 + 1)
    ndcg = dcg / idcg
    return ndcg

In [222]:
def get_HT(groundtruth, pred_rank_list,k):
    count = 0
    for pred in pred_rank_list:
        if count >= k:
            break
        if groundtruth[pred] == 1:
            return 1
        count += 1

    return 0

In [223]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [224]:
def merge_history(sum_history_test,test_key_set,training_sum_history_test,training_key_set,index,alpha):
    merged_history = {}
    for test_key_id in range(len(test_key_set)):
        test_key = test_key_set[test_key_id]
        test_history = sum_history_test[test_key]
        sum_training_history = np.zeros(len(test_history))
        for indecis in index[test_key_id]:
            training_key = training_key_set[indecis]
            sum_training_history += training_sum_history_test[training_key]

        sum_training_history = sum_training_history/len(index[test_key_id])

        merge = test_history*alpha + sum_training_history*(1-alpha)
        merged_history[test_key] = merge

    return merged_history

In [225]:
def merge_history_and_neighbors_future(future_data, sum_history_test, test_key_set,training_sum_history_test,
                                       training_key_set,index,alpha, beta):
    merged_history = {}
    for test_key_id in range(len(test_key_set)):
        test_key = test_key_set[test_key_id]
        test_history = sum_history_test[test_key]
        sum_training_history = np.zeros(len(test_history))
        sum_training_future = np.zeros(len(test_history))
        for indecis in index[test_key_id]:
            training_key = training_key_set[indecis]
            sum_training_history += training_sum_history_test[training_key]
            # future_vec = np.zeros((len(test_history)))
            for idx in future_data[training_key][1]:
                if idx >= 0:
                    sum_training_future[idx] += 1

        sum_training_history = sum_training_history/len(index[test_key_id])
        sum_training_future = sum_training_future/len(index[test_key_id])

        merge = (test_history*alpha + sum_training_history*(1-alpha))* beta + sum_training_future*(1-beta)
        merged_history[test_key] = merge

    return merged_history

In [226]:
def evaluate(data_chunk,  training_key_set, test_key_set, input_size, group_size,
             within_decay_rate, group_decay_rate, num_nearest_neighbors, alpha,  topk):
    activate_codes_num = -1
    temporal_decay_sum_history_training = temporal_decay_sum_history(data_chunk[training_chunk],
                                                                     training_key_set, input_size,
                                                                     group_size, within_decay_rate,
                                                                     group_decay_rate)
    temporal_decay_sum_history_test = temporal_decay_sum_history(data_chunk[training_chunk],
                                                                 test_key_set, input_size,
                                                                 group_size, within_decay_rate,
                                                                 group_decay_rate)
    index, distance = KNN(temporal_decay_sum_history_test, temporal_decay_sum_history_training,
                          num_nearest_neighbors)


    sum_history = merge_history(temporal_decay_sum_history_test, test_key_set, temporal_decay_sum_history_training,
                                training_key_set, index, alpha)


    if activate_codes_num < 0:
        # for i in range(1, 6):

        prec = []
        rec = []
        F = []
        prec1 = []
        rec1 = []
        F1 = []
        prec2 = []
        rec2 = []
        F2 = []
        prec3 = []
        rec3 = []
        F3 = []
        NDCG = []
        n_hit = 0

        num_ele = topk
        # print('k = ' + str(activate_codes_num))
        # evaluate(data_chunk, input_size,test_KNN_history, test_key_set, next_k_step)
        count = 0
        for iter in range(len(test_key_set)):
            # training_pair = training_pairs[iter - 1]
            # input_variable = training_pair[0]
            # target_variable = training_pair[1]
            input_variable = data_chunk[training_chunk][test_key_set[iter]]
            target_variable = data_chunk[test_chunk][test_key_set[iter]]

            if len(target_variable) < 2 + next_k_step:
                continue
            count += 1
            output_vectors = predict_with_elements_in_input(sum_history, test_key_set[iter])
            top = 400
            hit = 0
            for idx in range(len(output_vectors)):
                # for idx in [2]:

                output = np.zeros(input_size)
                target_topi = output_vectors[idx].argsort()[::-1][:top]
                c = 0
                for i in range(top):
                    if c >= num_ele:
                        break
                    output[target_topi[i]] = 1
                    c += 1

                vectorized_target = np.zeros(input_size)
                for ii in target_variable[1 + idx]:
                    vectorized_target[ii] = 1
                precision, recall, Fscore, correct = get_precision_recall_Fscore \
                    (vectorized_target, output)
                prec.append(precision)
                rec.append(recall)
                F.append(Fscore)
                if idx == 0:
                    prec1.append(precision)
                    rec1.append(recall)
                    F1.append(Fscore)
                elif idx == 1:
                    prec2.append(precision)
                    rec2.append(recall)
                    F2.append(Fscore)
                elif idx == 2:
                    prec3.append(precision)
                    rec3.append(recall)
                    F3.append(Fscore)
                hit += get_HT(vectorized_target, target_topi, num_ele)
                ndcg = get_NDCG1(vectorized_target, target_topi, num_ele)
                NDCG.append(ndcg)
            if hit == next_k_step:
                n_hit += 1


        # print('average precision of ' + ': ' + str(np.mean(prec)) + ' with std: ' + str(np.std(prec)))
        recall = np.mean(rec)
        ndcg = np.mean(NDCG)
        hr = n_hit / len(test_key_set)

    return recall, ndcg, hr

In [237]:
num_nearest_neighbors = 300#<---grid search
within_decay_rate = 0.9
group_decay_rate = 0.7
alpha = 0.2
group_size = 7
topk = 10
activate_codes_num = -1

In [238]:
files = ['TaFang_history_NB.csv', 'TaFang_future_NB.csv']

In [239]:
data_chunk, input_size, code_freq_at_first_claim = read_claim2vector_embedding_file_no_vector(files)

start dictionary generation...
{'MATERIAL_NUMBER': 11997}
# dimensions of final vector: 11997 | 0
finish dictionary generation*****


In [240]:
training_key_set, validation_key_set, test_key_set = partition_the_data_validate(data_chunk, list(data_chunk[test_chunk]), 1)

Number of training instances: 10000


In [241]:
temporal_decay_sum_history_training = temporal_decay_sum_history(data_chunk[training_chunk],training_key_set, input_size,group_size, within_decay_rate,group_decay_rate)

In [242]:
temporal_decay_sum_history_test = temporal_decay_sum_history(data_chunk[training_chunk],test_key_set, input_size,group_size, within_decay_rate,group_decay_rate)

In [243]:
index, distance = KNN(temporal_decay_sum_history_test, temporal_decay_sum_history_training,num_nearest_neighbors)

In [244]:
sum_history = merge_history(temporal_decay_sum_history_test, test_key_set, temporal_decay_sum_history_training,training_key_set, index, alpha)

In [ ]:
#if activate_codes_num < 0:
# for i in range(1, 6):
prec = []
rec = []
F = []
prec1 = []
rec1 = []
F1 = []
prec2 = []
rec2 = []
F2 = []
prec3 = []
rec3 = []
F3 = []
NDCG = []
n_hit = 0
output_vec_list=[]
vec_target_list=[]

num_ele = topk
# print('k = ' + str(activate_codes_num))
# evaluate(data_chunk, input_size,test_KNN_history, test_key_set, next_k_step)
count = 0

In [265]:
for iter in range(len(test_key_set)):
    # training_pair = training_pairs[iter - 1]
    # input_variable = training_pair[0]
    # target_variable = training_pair[1]
    input_variable = data_chunk[training_chunk][test_key_set[iter]]
    target_variable = data_chunk[test_chunk][test_key_set[iter]]

    if len(target_variable) < 2 + next_k_step:
        continue
        
    count += 1
    output_vectors = predict_with_elements_in_input(sum_history, test_key_set[iter])
    #test_output_vectors.append(output_vectors)
    top = 400
    hit = 0
    for idx in range(len(output_vectors)):
        # for idx in [2]:
        output = np.zeros(input_size)
        target_topi = output_vectors[idx].argsort()[::-1][:top]
        c = 0
        for i in range(top):
            if c >= num_ele:
                break
            output[target_topi[i]] = 1
            c += 1

        vectorized_target = np.zeros(input_size)
        for ii in target_variable[1 + idx]:
            vectorized_target[ii] = 1
        
        output_vec_list.append(output)
        vec_target_list.append(vectorized_target)
        
        precision, recall, Fscore, correct = get_precision_recall_Fscore(vectorized_target, output)
        prec.append(precision)
        rec.append(recall)
        F.append(Fscore)
        
        if idx == 0:
            prec1.append(precision)
            rec1.append(recall)
            F1.append(Fscore)
            
        elif idx == 1:
            prec2.append(precision)
            rec2.append(recall)
            F2.append(Fscore)
            
        elif idx == 2:
            prec3.append(precision)
            rec3.append(recall)
            F3.append(Fscore)
            
        hit += get_HT(vectorized_target, target_topi, num_ele)
        ndcg = get_NDCG1(vectorized_target, target_topi, num_ele)
        NDCG.append(ndcg)
        
    if hit == next_k_step:
        n_hit += 1

# print('average precision of ' + ': ' + str(np.mean(prec)) + ' with std: ' + str(np.std(prec)))
recall = np.mean(rec)
ndcg = np.mean(NDCG)
hr = n_hit / len(test_key_set)

In [272]:
len(output_vectors)

1

In [270]:
len(output_vec_list)

2778

In [269]:
vec_target_list

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 

In [257]:
test_output_vectors

[[array([4.16571429e-05, 6.32773828e-04, 1.72186884e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05, 2.99003403e-04, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05, 4.40186412e-04, 1.72186884e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05, 9.69389848e-04, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05, 4.56458073e-04, 1.72186884e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05, 5.59275672e-04, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05, 4.41634001e-04, 1.72186884e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05, 4.48575222e-04, 1.72186884e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])],
 [array([4.16571429e-05,

In [250]:
data_chunk[training_chunk][test_key_set[0]]

[[-1],
 array([ 629, 1588, 3937]),
 array([    6,    24,   412,   469,  5271,  6090,  6387,  6458, 10329]),
 array([   6,   93,  256,  336,  368,  374,  493,  521,  690,  747, 1440,
        1565, 1879, 1971, 2343, 2751, 3347, 3453, 4481, 5417, 5872, 6627,
        9267]),
 [-1]]

In [255]:
test_key_set[0]

'11149'

In [253]:
data_chunk[training_chunk][training_key_set[0]]

[[-1],
 array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
        67]),
 array([68]),
 array([50, 69]),
 array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79]),
 array([70, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91]),
 array([  4,  50,  61,  66,  68,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102]),
 array([ 68, 103, 104]),
 array([ 66,  70, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120]),
 array([121]),
 array([92]),
 array([ 54, 122, 123]),
 array([124, 125]),
 array([126, 127]),
 array([ 92, 106, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
        139, 140, 141]),
 array([ 52, 106, 142, 143, 144, 145, 146, 147]),
 [-1]]

In [194]:
print('Num. of top: ', topk)
recall, ndcg, hr = evaluate(data_chunk, training_key_set, test_key_set, input_size,
                            group_size, within_decay_rate, group_decay_rate,
                            num_nearest_neighbors, alpha,  topk)

print('recall: ', str(recall))
print('NDCG: ', str(ndcg))

Num. of top:  20
recall:  0.21378931797550685
NDCG:  0.2624959810199685


__________________

In [54]:
#!conda install numpy -y

In [195]:
import sys
from sklearn.neighbors import NearestNeighbors
#from __future__ import unicode_literals, print_function, division
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import sys
import math
import csv

sys.maxsize
sys.maxsize
activate_codes_num = -1
next_k_step = 1
training_chunk = 0
test_chunk = 1

num_nearest_neighbors = 300#k (num_nearest_neighbors) chosen [100, 300, 500, 700, 900, 1100,1300]
within_decay_rate = 0.9# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
group_decay_rate = 0.7# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
alpha = 0.7
group_size = 7
topk = 20


In [196]:
files = ['TaFang_history_NB.csv', 'TaFang_future_NB.csv']


#attributes_list = ['DRG', 'PROVCAT ', 'RVNU_CD', 'DIAG', 'PROC']
attributes_list = ['MATERIAL_NUMBER']
path = ''
print('start dictionary generation...')

#dictionary_table, num_dim, counter_table = generate_dictionary_BA(files, attributes_list)############################
dictionary_table = {}
counter_table = {}

for attr in attributes_list:
    dictionary = {}
    dictionary_table[attr] = dictionary
    counter_table[attr] = 0


#csv.field_size_limit(sys.maxsize)
for filename in files:
    count = 0
    with open(path + filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            if count == 0:
                count += 1
                continue
            key = attributes_list[0]
            if row[2] not in dictionary_table[key]:
                dictionary_table[key][row[2]] = counter_table[key]
                counter_table[key] = counter_table[key] + 1
                count += 1
                
print(counter_table)
total = 0
for key in counter_table.keys():
    total = total + counter_table[key]
    
print('# dimensions of final vector: ' + str(total) + ' | '+str(count-1))
dictionary_table, num_dim, counter_table = dictionary_table, total, counter_table

print('finish dictionary generation*****')
usr_attr = 'CUSTOMER_ID'
ord_attr = 'ORDER_NUMBER'

freq_max = 200
## all the follow three ways array. First index is patient, second index is the time step, third is the feature vector
data_chunk = []
day_gap_counter = []
claims_counter = 0
num_claim = 0
code_freq_at_first_claim = np.zeros(num_dim+2)


start dictionary generation...
{'MATERIAL_NUMBER': 11997}
# dimensions of final vector: 11997 | 0
finish dictionary generation*****


In [197]:
for file_id in range(len(files)):
    count = 0
    data_chunk.append({})
    filename = files[file_id]
    with open(path + filename, 'r') as csvfile:
        #gap_within_one_year = np.zeros(365)
        reader = csv.DictReader(csvfile)
        last_pid_date = '*'
        last_pid = '-1'
        last_days = -1
        # 2 more elements in the end for start and end states
        feature_vector = []
        for row in reader:
            cur_pid_date = row[usr_attr] + '_' + row[ord_attr]
            cur_pid = row[usr_attr]
            #cur_days = int(row[ord_attr])

            if cur_pid != last_pid:
                # start state
                tmp = [-1]
                data_chunk[file_id][cur_pid] = []
                data_chunk[file_id][cur_pid].append(tmp)
                num_claim = 0

            if cur_pid_date not in last_pid_date:
                if last_pid_date not in '*' and last_pid not in '-1':
                    sorted_feature_vector = np.sort(feature_vector)
                    data_chunk[file_id][last_pid].append(sorted_feature_vector)
                    if len(sorted_feature_vector) > 0:
                        count = count + 1
                    #data_chunk[file_id][last_pid].append(feature_vector)
                feature_vector = []
                claims_counter = 0
                
            if cur_pid != last_pid:
                # end state
                if last_pid not in '-1':
                    tmp = [-1]
                    data_chunk[file_id][last_pid].append(tmp)

            key = attributes_list[0]
            within_idx = dictionary_table[key][row[key]]
            previous_idx = 0

            for j in range(attributes_list.index(key)):
                previous_idx = previous_idx + counter_table[attributes_list[j]]
            idx = within_idx + previous_idx

            # set corresponding dimention to 1
            if idx not in feature_vector:
                feature_vector.append(idx)

            last_pid_date = cur_pid_date
            last_pid = cur_pid
            #last_days = cur_days
            if file_id == 1:
                claims_counter = claims_counter + 1
                
        if last_pid_date not in '*' and last_pid not in '-1':
            data_chunk[file_id][last_pid].append(np.sort(feature_vector))
            
            

In [198]:
data_chunk, input_size, code_freq_at_first_claim = data_chunk, num_dim + 2, code_freq_at_first_claim
data_chunk, key_set, next_k_step=data_chunk, list(data_chunk[test_chunk]), 1

filtered_key_set = []
past_chunk = 0
future_chunk = 1
for key in key_set:
    if len(data_chunk[past_chunk][key]) <= 3:
        continue
    if len(data_chunk[future_chunk][key]) < 2 + next_k_step:
        continue
    filtered_key_set.append(key)

training_key_set = filtered_key_set[0:int(4 / 5 * len(filtered_key_set)*0.9)]
validation_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)*0.9):int(4 / 5 * len(filtered_key_set))]
print('Number of training instances: ' + str(len(training_key_set)))
test_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)):]
training_key_set, validation_key_set, test_key_set = training_key_set, validation_key_set, test_key_set

Number of training instances: 10000


In [199]:
files = ['Dunnhumby_history.csv', 'Dunnhumby_future.csv']
df=pd.read_csv('TaFang_history_NB.csv')
#list of features
attributes_list = ['MATERIAL_NUMBER']


dictionary_table={'MATERIAL_NUMBER':dict(zip(map(str,list(set(df['MATERIAL_NUMBER']))),df.index))}
counter_table={'MATERIAL_NUMBER':len(set(df['MATERIAL_NUMBER']))}
num_dim=len(set(df['MATERIAL_NUMBER']))

dictionary_table, num_dim, counter_table = dictionary_table, total, counter_table

In [200]:
sum_history = {}
for key in key_set[:5]:

    vec_list = data_set[key]
    num_vec = len(vec_list)-2
    his_list = []

    #removes the [-1] flags in data
    for idx in range(1,num_vec+1):
                    #output_size=11999
        his_vec = np.zeros(output_size)
        # np.power(.9,15-iterater)  #counts down 
        decayed_val = np.power(within_decay_rate,num_vec-idx)
        #elements in data_set['2']
        for ele in vec_list[idx]:
            his_vec[ele] = decayed_val
        his_list.append(his_vec)

    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        grouped_list,real_group_size = grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:
            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        grouped_list,real_group_size = grouped_vec_list, group_size

    his_vec = np.zeros(output_size)
    for idx in range(real_group_size):
        decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
        if idx>=len(grouped_list):
            print( 'idx: '+ str(idx))
            print('len(grouped_list): ' + str(len(grouped_list)))
        his_vec += grouped_list[idx]*decayed_val
    sum_history[key] = his_vec/real_group_size
    # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)'''

In [117]:
import sys
from sklearn.neighbors import NearestNeighbors
#from __future__ import unicode_literals, print_function, division
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import sys
import math
import csv

sys.maxsize
sys.maxsize
activate_codes_num = -1
next_k_step = 1
training_chunk = 0
test_chunk = 1

num_nearest_neighbors = 300#k (num_nearest_neighbors) chosen [100, 300, 500, 700, 900, 1100,1300]
within_decay_rate = 0.9# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
group_decay_rate = 0.7# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
alpha = 0.7
group_size = 7
topk = 10

files = ['TaFang_history_NB.csv', 'TaFang_future_NB.csv']
#attributes_list = ['MATERIAL_NUMBER']
attributes_list = 'MATERIAL_NUMBER'
usr_attr = 'CUSTOMER_ID'
ord_attr = 'ORDER_NUMBER'

freq_max = 200
## all the follow three ways array. First index is patient, second index is the time step, third is the feature vector
data_chunk = []
day_gap_counter = []
claims_counter = 0
num_claim = 0

filtered_key_set = []
past_chunk = 0
future_chunk = 1

In [118]:
df1=pd.read_csv(files[0])
tmp1=df1.sort_values([usr_attr,ord_attr]).groupby([usr_attr,ord_attr])[attributes_list].apply(np.array).reset_index()
tmp1=tmp1.groupby([usr_attr])[attributes_list].apply(list).reset_index()
tmp1[attributes_list]=tmp1[attributes_list].apply(lambda x: [[-1]]+[np.array(sorted(i)) for i in x]+[[-1]])

In [119]:
df2=pd.read_csv(files[1])
tmp2=df2.sort_values([usr_attr,ord_attr]).groupby([usr_attr,ord_attr])[attributes_list].apply(np.array).reset_index()
tmp2=tmp2.groupby([usr_attr])[attributes_list].apply(list).reset_index()
tmp2[attributes_list]=tmp2[attributes_list].apply(lambda x: [[-1]]+[np.array(sorted(i)) for i in x]+[[-1]])

In [120]:
history_key_set=tmp1.assign(MATERIAL_NUMBER=np.where(tmp1.MATERIAL_NUMBER.apply(len)<= 3,None,tmp1.MATERIAL_NUMBER)).assign(CUSTOMER_ID=tmp1.CUSTOMER_ID.apply(str))
history_key_set=list(history_key_set[~history_key_set.MATERIAL_NUMBER.isnull()]['CUSTOMER_ID'])

future_key_set=tmp2.assign(MATERIAL_NUMBER=np.where(tmp2.MATERIAL_NUMBER.apply(len)< 2 + next_k_step,None,tmp2.MATERIAL_NUMBER)).assign(CUSTOMER_ID=tmp2.CUSTOMER_ID.apply(str))
future_key_set=list(future_key_set[~future_key_set.MATERIAL_NUMBER.isnull()]['CUSTOMER_ID'])

filtered_key_set=[x for x in history_key_set if x in future_key_set]
#slice filtered_key_set range(0:10001)
training_key_set = filtered_key_set[0:int(4 / 5 * len(filtered_key_set)*0.9)]
print('Number of training instances: ' + str(len(training_key_set)))
#slice filtered_key_set range(10001:11112)
validation_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)*0.9):int(4 / 5 * len(filtered_key_set))]
#slice filtered_key_set range(:11112)
test_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)):]

Number of training instances: 10001


In [121]:
num_dim=len(set(df1[attributes_list]))
code_freq_at_first_claim = np.zeros(num_dim+2)

data_chunk=[dict(zip(tmp1[usr_attr].apply(str),tmp1[attributes_list])),dict(zip(tmp2[usr_attr].apply(str),tmp2[attributes_list]))]

In [122]:
#data_chunk=data_chunk
input_size=num_dim + 2
#code_freq_at_first_claim=code_freq_at_first_claim 
#data_chunk=data_chunk
key_set=list(data_chunk[test_chunk]),
next_k_step=1

In [123]:
data_set=data_chunk[training_chunk]
key_set=training_key_set 
output_size=input_size
#within_decay_rate=within_decay_rate
#group_decay_rate=group_decay_rate
print('Num. of top: ', topk)

Num. of top:  10


In [124]:
def decay_history(num_vec,within_decay_rate):
    #removes the [-1] flags in data
    for idx in range(1,num_vec+1):
        his_vec = np.zeros(output_size)#output_size=11999
        # np.power(.9,15-iterater)  #counts down 
        decayed_val = np.power(within_decay_rate,num_vec-idx)
        #elements in data_set['2']
        for ele in vec_list[idx]:
            his_vec[ele] = decayed_val
        his_list.append(his_vec)

In [125]:
sum_history = {}
for key in key_set:
    vec_list = data_set[key]
    num_vec = len(vec_list)-2
    his_list = []
    
    decay_history(num_vec,within_decay_rate)
        
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        grouped_list,real_group_size = grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:
            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        grouped_list,real_group_size = grouped_vec_list, group_size

    his_vec = np.zeros(output_size)
    for idx in range(real_group_size):
        decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
        if idx>=len(grouped_list):
            print( 'idx: '+ str(idx))
            print('len(grouped_list): ' + str(len(grouped_list)))
        his_vec += grouped_list[idx]*decayed_val
    sum_history[key] = his_vec/real_group_size
    # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)

In [126]:
temporal_decay_sum_history_training = sum_history

In [127]:
data_set=data_chunk[training_chunk] 
key_set=test_key_set 
output_size=input_size
#group_size=group_size
#within_decay_rate=within_decay_rate
#group_decay_rate=group_decay_rate
#data_set, key_set, output_size,group_size,within_decay_rate,group_decay_rate=data_chunk[training_chunk],test_key_set, input_size,group_size, within_decay_rate,group_decay_rate

In [128]:
sum_history = {}
for key in key_set:
    vec_list = data_set[key]
    num_vec = len(vec_list) - 2
    his_list = []
    
    decay_history(num_vec,within_decay_rate)
        
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        grouped_list,real_group_size = grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:
            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        grouped_list,real_group_size = grouped_vec_list, group_size
    his_vec = np.zeros(output_size)
    for idx in range(real_group_size):
        decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
        if idx>=len(grouped_list):
            print( 'idx: '+ str(idx))
            print('len(grouped_list): ' + str(len(grouped_list)))
        his_vec += grouped_list[idx]*decayed_val
    sum_history[key] = his_vec/real_group_size
    # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)

In [129]:
temporal_decay_sum_history_test = sum_history

In [130]:
query_set=temporal_decay_sum_history_test 
target_set=temporal_decay_sum_history_training 
k=num_nearest_neighbors

In [131]:
#def KNN(query_set, target_set, k):
history_mat = []
for key in target_set.keys():
    history_mat.append(target_set[key])
    
test_mat = []
for key in query_set.keys():
    test_mat.append(query_set[key])
print('Finding k nearest neighbors...')

nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(history_mat)
distances, indices = nbrs.kneighbors(test_mat)
print('Finish KNN search.' )
index, distance = indices,distances


Finding k nearest neighbors...
Finish KNN search.


In [132]:

sum_history_test=temporal_decay_sum_history_test
#test_key_set=test_key_set
training_sum_history_test=temporal_decay_sum_history_training
#training_key_set=training_key_set
#index=index
#alpha=alpha 

In [133]:
merged_history = {}
for test_key_id in range(len(test_key_set)):
    test_key = test_key_set[test_key_id]
    test_history = sum_history_test[test_key]
    sum_training_history = np.zeros(len(test_history))
    for indecis in index[test_key_id]:
        training_key = training_key_set[indecis]
        sum_training_history += training_sum_history_test[training_key]

    sum_training_history = sum_training_history/len(index[test_key_id])

    merge = test_history*alpha + sum_training_history*(1-alpha)
    merged_history[test_key] = merge

sum_history = merged_history

In [163]:
activate_codes_num

-1

In [153]:
if activate_codes_num < 0:
    # for i in range(1, 6):
    prec = []
    rec = []
    F = []
    prec1 = []
    rec1 = []
    F1 = []
    prec2 = []
    rec2 = []
    F2 = []
    prec3 = []
    rec3 = []
    F3 = []
    NDCG = []
    n_hit = 0

    num_ele = topk
    # print('k = ' + str(activate_codes_num))
    # evaluate(data_chunk, input_size,test_KNN_history, test_key_set, next_k_step)
    count = 0
    for iter in range(len(test_key_set)):
        # training_pair = training_pairs[iter - 1]
        # input_variable = training_pair[0]
        # target_variable = training_pair[1]
        input_variable = data_chunk[training_chunk][test_key_set[iter]]
        target_variable = data_chunk[test_chunk][test_key_set[iter]]

        if len(target_variable) < 2 + next_k_step:
            continue
        count += 1
        sum_history,key = sum_history, test_key_set[iter]
        #def predict_with_elements_in_input(sum_history,key):
        output_vectors = []

        for idx in range(next_k_step):
            vec = sum_history[key]
            output_vectors.append(vec)
        #output_vectors = output_vectors        
        top = 400
        hit = 0
        for idx in range(len(output_vectors)):
            # for idx in [2]:
            output = np.zeros(input_size)
            target_topi = output_vectors[idx].argsort()[::-1][:top]
            c = 0
            for i in range(top):
                if c >= num_ele:
                    break
                output[target_topi[i]] = 1
                c += 1

            vectorized_target = np.zeros(input_size)
            for ii in target_variable[1 + idx]:
                vectorized_target[ii] = 1
            groundtruth,pred = vectorized_target, output
            #def get_precision_recall_Fscore(groundtruth,pred):
            a = groundtruth
            b = pred
            correct = 0
            truth = 0
            positive = 0

            for idx in range(len(a)):
                if a[idx] == 1:
                    truth += 1
                    if b[idx] == 1:
                        correct += 1
                if b[idx] == 1:
                    positive += 1

            flag = 0
            if 0 == positive:
                precision = 0
                flag = 1
                #print('postivie is 0')
            else:
                precision = correct/positive
            if 0 == truth:
                recall = 0
                flag = 1
                #print('recall is 0')
            else:
                recall = correct/truth

            if flag == 0 and precision + recall > 0:
                G = 2*precision*recall/(precision+recall)
            else:
                G = 0
            precision, recall, Fscore, correct = precision, recall, G, correct            
            prec.append(precision)
            rec.append(recall)
            F.append(Fscore)
            if idx == 0:
                prec1.append(precision)
                rec1.append(recall)
                F1.append(Fscore)
            elif idx == 1:
                prec2.append(precision)
                rec2.append(recall)
                F2.append(Fscore)
            elif idx == 2:
                prec3.append(precision)
                rec3.append(recall)
                F3.append(Fscore)
            groundtruth, pred_rank_list,k=vectorized_target, target_topi, num_ele
            #def get_HT(groundtruth, pred_rank_list,k):
            count = 0
            for pred in pred_rank_list:
                if count >= k:
                    break
                if groundtruth[pred] == 1:
                    hit += 1
                count += 1
            hit += 0
            #input_size = 100
            vectorized_target, target_topi, num_ele=groundtruth, pred_rank_list,k
            #def get_NDCG1(groundtruth, pred_rank_list,k):
            count = 0
            dcg = 0
            for pred in pred_rank_list:
                if count >= k:
                    break
                if groundtruth[pred] == 1:
                    dcg += (1)/math.log2(count+1+1)
                count += 1
            idcg = 0
            num_real_item = np.sum(groundtruth)
            num_item = int(num_real_item)
            for i in range(num_item):
                idcg += (1) / math.log2(i + 1 + 1)
            ndcg = dcg / idcg
            NDCG.append(ndcg)
        if hit == next_k_step:
            n_hit += 1
            
    print('average precision of ' + ': ' + str(np.mean(prec)) + ' with std: ' + str(np.std(prec)))
    recall = np.mean(rec)
    ndcg = np.mean(NDCG)
    hr = n_hit / len(test_key_set)
    
#recall, ndcg, hr = recall, ndcg, hr
print('recall: ', str(recall))
print('NDCG: ', str(ndcg))

average precision of : 0.053364519611371 with std: 0.07979832834575636
recall:  0.1296420382929529
NDCG:  0.09875611529474698


In [154]:
precision

0.0

In [155]:
recall

0.1296420382929529

In [156]:
Fscore

0

In [157]:
correct

0

In [137]:
pred

4739

In [138]:
output

array([0., 0., 0., ..., 0., 0., 0.])

In [139]:
len(groundtruth)

11999

In [140]:
pred

4739

In [141]:
n_hit

770

In [142]:
precision

0.0

In [143]:
recall

0.1296420382929529

In [144]:
G

0

In [159]:
print('finish dictionary generation*****')
usr_attr = 'CUSTOMER_ID'
ord_attr = 'ORDER_NUMBER'

freq_max = 200
## all the follow three ways array. First index is patient, second index is the time step, third is the feature vector
data_chunk = []
day_gap_counter = []
claims_counter = 0
num_claim = 0
code_freq_at_first_claim = np.zeros(num_dim+2)


finish dictionary generation*****


In [287]:
#with open('TaFang_history_NB.csv', 'r') as csvfile:
#    #gap_within_one_year = np.zeros(365)
#    reader = csv.DictReader(csvfile)
#    for row in reader:
#        print(row)

In [203]:
'''for file_id in range(len(files)):
    count = 0
    data_chunk.append({})
    filename = files[file_id]
    with open(path + filename, 'r') as csvfile:
        #gap_within_one_year = np.zeros(365)
        reader = csv.DictReader(csvfile)
        last_pid_date = '*'
        last_pid = '-1'
        last_days = -1
        # 2 more elements in the end for start and end states
        feature_vector = []
        for row in reader:
            cur_pid_date = row['CUSTOMER_ID'] + '_' + row['ORDER_NUMBER']
            cur_pid = row['CUSTOMER_ID']
            #cur_days = int(row[ord_attr]) 
            
            if cur_pid != last_pid:
                # start state
                tmp = [-1]
                data_chunk[file_id][cur_pid] = []
                data_chunk[file_id][cur_pid].append(tmp)
                num_claim = 0
                
            if cur_pid_date not in last_pid_date:
                if last_pid_date not in '*' and last_pid not in '-1':
                    sorted_feature_vector = np.sort(feature_vector)
                    data_chunk[file_id][last_pid].append(sorted_feature_vector)
                    if len(sorted_feature_vector) > 0:
                        count = count + 1
                    #data_chunk[file_id][last_pid].append(feature_vector)
                feature_vector = []
                claims_counter = 0
                
            if cur_pid != last_pid:
                # end state
                if last_pid not in '-1':
                    tmp = [-1]
                    data_chunk[file_id][last_pid].append(tmp)

            key = attributes_list[0]
            within_idx = dictionary_table[key][row[key]]
            previous_idx = 0

            for j in range(attributes_list.index(key)):
                previous_idx = previous_idx + counter_table[attributes_list[j]]
            idx = within_idx + previous_idx

            # set corresponding dimention to 1
            if idx not in feature_vector:
                feature_vector.append(idx)

            last_pid_date = cur_pid_date
            last_pid = cur_pid
            #last_days = cur_days
            if file_id == 1:
                claims_counter = claims_counter + 1
                
        if last_pid_date not in '*' and last_pid not in '-1':
            data_chunk[file_id][last_pid].append(np.sort(feature_vector))'''

In [289]:
len(code_freq_at_first_claim)

11999

In [123]:
import pandas as pd
df=pd.read_csv('TaFang_history_NB.csv',sep=',')
df[df.CUSTOMER_ID==2].head(20)

,CUSTOMER_ID,ORDER_NUMBER,MATERIAL_NUMBER
56,2,1,51
57,2,1,52
58,2,1,53
59,2,1,54
60,2,1,55
61,2,1,56
62,2,1,57
63,2,1,58
64,2,1,59
65,2,1,60


In [139]:
data_chunk[0]['2']

[[-1],
 array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
        67]),
 array([68]),
 array([50, 69]),
 array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79]),
 array([70, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91]),
 array([  4,  50,  61,  66,  68,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102]),
 array([ 68, 103, 104]),
 array([ 66,  70, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120]),
 array([121]),
 array([92]),
 array([ 54, 122, 123]),
 array([124, 125]),
 array([126, 127]),
 array([ 92, 106, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
        139, 140, 141]),
 array([ 52, 106, 142, 143, 144, 145, 146, 147]),
 [-1]]

In [134]:
tmp=df.groupby(['CUSTOMER_ID','ORDER_NUMBER'])['MATERIAL_NUMBER'].apply(np.array)

In [135]:
tmp=tmp.reset_index()

In [136]:
tmp.MATERIAL_NUMBER.apply(lambda x: len(x)).mean()

6.220074789418682

In [141]:
tmp=df.groupby(['CUSTOMER_ID','ORDER_NUMBER'])['MATERIAL_NUMBER'].apply(np.array).reset_index()
tmp=tmp.groupby(['CUSTOMER_ID'])['MATERIAL_NUMBER'].apply(list).reset_index()
data_chunk_mimic=dict(zip(tmp.CUSTOMER_ID.apply(str),tmp.MATERIAL_NUMBER))

In [142]:
data_chunk_mimic['2']

[array([51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68], dtype=int64),
 array([69], dtype=int64),
 array([51, 70], dtype=int64),
 array([71, 72, 73, 74, 75, 76, 77, 78, 79, 80], dtype=int64),
 array([81, 82, 83, 84, 85, 71, 86, 87, 88, 89, 90, 91, 92], dtype=int64),
 array([ 93,  62,  67,  94,  95,  51,  69,  96,  97,  98,  99, 100, 101,
        102,   5, 103], dtype=int64),
 array([104,  69, 105], dtype=int64),
 array([ 71, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118,  67, 119, 120, 121], dtype=int64),
 array([122], dtype=int64),
 array([93], dtype=int64),
 array([123,  55, 124], dtype=int64),
 array([125, 126], dtype=int64),
 array([127, 128], dtype=int64),
 array([129, 130, 107, 131,  93, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142], dtype=int64),
 array([143, 144, 145, 146, 107,  53, 147, 148], dtype=int64)]

Looks like its CUSTOMER_ID grouped by MATERIAL_NUMBER

In [49]:
#tmp_chunk=list(pd.Series(data_chunk[0]['2'][1:-1]).explode())
#testing_chunk=df[(df.CUSTOMER_ID==2)]
#testing_chunk[~testing_chunk.MATERIAL_NUMBER.isin(tmp_chunk)]

In [50]:
#list(pd.Series(data_chunk[0]['2'][1:-1]).explode())

In [128]:
df_dict=df.groupby('ORDER_NUMBER')['MATERIAL_NUMBER'].apply(list).apply(sorted).reset_index()
df_dict=dict(zip(df_dict.ORDER_NUMBER,df_dict.MATERIAL_NUMBER))

In [129]:
tmp=df.groupby('CUSTOMER_ID')['ORDER_NUMBER'].apply(set).apply(list).reset_index().merge(df.groupby('CUSTOMER_ID')['MATERIAL_NUMBER'].apply(list).reset_index(),how='left',on='CUSTOMER_ID')

In [130]:
tmp

,CUSTOMER_ID,ORDER_NUMBER,MATERIAL_NUMBER
0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 7,..."
1,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 6..."
2,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[149, 150, 123, 151, 152, 153, 152, 154, 150, ..."
3,4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]","[174, 175, 176, 177, 178, 179, 180, 181, 182, ..."
4,5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]","[228, 229, 230, 51, 231, 232, 233, 234, 235, 2..."
...,...,...,...
13944,13945,"[1, 2]","[51, 7597, 369, 4114, 1587, 8781, 4020, 1566]"
13945,13946,"[1, 2]","[1497, 884, 5175, 51, 871]"
13946,13947,"[1, 2]","[114, 9997, 389, 8666, 1428]"
13947,13948,"[1, 2]","[1428, 51, 4292, 818, 427, 668, 536, 1218]"


In [132]:
#tmp['ORDERS']=tmp.ORDER_NUMBER.apply(lambda x: list(map(df_dict.get,x)))#.apply(set)
#tmp['ORDERS']=tmp['ORDERS'].apply(lambda x: [list(set(i)) for i in x])

In [ ]:
tmp

In [54]:
tmp.ORDER_NUMBER.apply(lambda x: len(x))

0        13
1        15
2         9
3        14
4        14
         ..
13944     2
13945     2
13946     2
13947     2
13948     2
Name: ORDER_NUMBER, Length: 13949, dtype: int64

In [80]:
tmp.ORDERS[0]

[[1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  